In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time 


In [2]:
options = Options()
options.add_argument("--start-maximized")

service = Service()

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.kayak.com/flights")
time.sleep(5)
print(driver.title)


Cheap Flights, Airline Tickets & Airfare Deals | KAYAK


In [3]:
departure_city = "New York"
destination_city = "London"
departure_date = "April 4, 2025"
return_date = "April 5, 2025"

# Fill departure city

In [4]:

departure_input = driver.find_element(By.XPATH, "//input[@aria-label='Flight origin input']")
departure_input

<selenium.webdriver.remote.webelement.WebElement (session="0cd83a51a5448131f0670be2c8c6e63f", element="f.74D8A6A103BAF09CF42DED1DA60407B6.d.6536020B48848C3F233DFAC5EA0FCE7C.e.15")>

In [5]:
departure_input.click()
time.sleep(2)

In [6]:
departure_input.send_keys(Keys.CONTROL + "a")  # select all text
departure_input.send_keys(Keys.BACKSPACE)
departure_input.send_keys(Keys.BACKSPACE)
departure_input.send_keys(Keys.BACKSPACE)

In [7]:
departure_input.send_keys(departure_city)
time.sleep(2)

In [8]:
departure_input.send_keys(Keys.ENTER)

# Fill destination city

In [ ]:
destination_input = driver.find_element(By.XPATH, "//input[@aria-label='Flight destination input']")
destination_input.click()
time.sleep(2)
destination_input.send_keys(Keys.CONTROL + "a")
destination_input.send_keys(Keys.BACKSPACE)
destination_input.send_keys(Keys.BACKSPACE)
destination_input.send_keys(Keys.BACKSPACE)
destination_input.send_keys(destination_city)
time.sleep(2)
destination_input.send_keys(Keys.ENTER)
time.sleep(5)

In [11]:
departure_date_xpath = f"//div[@role='button' and contains(@aria-label, '{departure_date}')]"

departure_date_element = driver.find_element(By.XPATH, departure_date_xpath)
departure_date_element.click()

In [12]:
departure_date_xpath = f"//div[@role='button' and contains(@aria-label, '{return_date}')]"

departure_date_element = driver.find_element(By.XPATH, departure_date_xpath)
departure_date_element.click()
time.sleep(2)

In [13]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Search']"))
)
search_button.click()
time.sleep(10)

In [ ]:
# Store the current window handle
main_window = driver.current_window_handle

# After clicking search button, wait a second or two for new tab to open
time.sleep(2)

# Get all open windows/tabs
all_windows = driver.window_handles

# Switch to the new tab (the one that is not the main window)
for window in all_windows:
    if window != main_window:
        driver.switch_to.window(window)
        break

In [15]:
flights = driver.find_elements(By.XPATH, "//div[contains(@id,'flight-results-list-wrapper')]")

flights

[<selenium.webdriver.remote.webelement.WebElement (session="0cd83a51a5448131f0670be2c8c6e63f", element="f.06CD705FBE25A159454FC4EC05C5D11E.d.B8E5E92123921FF3F6D8D5B1060DAAF7.e.412")>]

In [16]:
len(flights)

1

In [17]:
'''price = flights[0].find_element(By.XPATH, ".//span[contains(@class,'price-text')]").text'''

'price = flights[0].find_element(By.XPATH, ".//span[contains(@class,\'price-text\')]").text'

In [18]:
flight_combos = driver.find_elements(By.XPATH, "//ol[contains(@class, 'hJSA-list')]")
len(flight_combos)

16

In [19]:
flight_combos = driver.find_elements(By.XPATH, "//ol[contains(@class, 'hJSA-list')]")
len(flight_combos)


16

In [ ]:
load_more_clicks = 5
for i in range(load_more_clicks):
    try:
        load_more = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(text(),'Show more results')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", load_more)
        time.sleep(1)
        load_more.click()
        print(f"Clicked Load More {i+1} times")
        time.sleep(5)  # Allow time for new results to load
    except:
        print("No more 'Show more results' button found.")
        break

Clicked Load More 1 times
Clicked Load More 2 times
Clicked Load More 3 times
No more 'Show more results' button found.


In [21]:
flight_results = driver.find_elements(By.XPATH, "//div[contains(@class, 'nrc6-wrapper')]")
len(flight_results)

32

In [22]:
from bs4 import BeautifulSoup
for i,combo in enumerate(flight_results):
    
    flights = combo.find_elements(By.XPATH, ".//li[contains(@class,'hJSA-item')]")
    '''
    for i, el in enumerate(flights, start=1):
        html_code = el.get_attribute('outerHTML')
        soup = BeautifulSoup(html_code, 'html.parser')
        pretty_html = soup.prettify()
        print(f"\nElement {i}:\n{pretty_html}\n{'='*40}")
    '''
    print(i)
    price=combo.find_elements(By.XPATH, ".//div[contains(@class,'price-section')]//div[contains(@class, 'price-text')]")[0].text
    print(price)
    for flight in flights:
        # departure & arrival times
        times = flight.find_elements(
            By.XPATH, ".//div[contains(@class,'VY2U')]//span[contains(text(), 'am') or contains(text(), 'pm')]"
        )
        dep_time = times[0].text.strip().split('+')[0].strip()
        arr_time = times[1].text.strip().split('+')[0].strip()
        print(dep_time,arr_time)
        # airline
       
        airline = flight.find_element(
            By.XPATH, ".//div[contains(@class,'c_cgF') and not(.//span)]"
        ).text.strip()

        # duration
        duration = flight.find_element(
            By.XPATH, ".//div[contains(text(),'h') and contains(text(),'m')]"
        ).text.strip()

        # origin and destination airports
        airports = flight.find_elements(
            By.XPATH, ".//div[contains(@class,'EFvI')]//div[contains(@class,'c_cgF c_cgF-mod-variant-default')]"
        )
        origin = airports[0].get_attribute("title")
        destination = airports[1].get_attribute("title")
        
        '''for i, el in enumerate(airports, start=1):
            html_code = el.get_attribute('outerHTML')
            soup = BeautifulSoup(html_code, 'html.parser')
            pretty_html = soup.prettify()
            print(f"\nElement {i}:\n{pretty_html}\n{'='*40}")'''

        print(f"Departure: {dep_time}, Arrival: {arr_time}, Airline: {airline}, Duration: {duration}")
        print(f"From: {origin}  -->  To: {destination}\n")
       

       

0
$959
9:45 am 9:40 pm
Departure: 9:45 am, Arrival: 9:40 pm, Airline: British Airways, Duration: 6h 55m
From: New York John F Kennedy Intl  -->  To: London Heathrow

7:05 pm 10:00 pm
Departure: 7:05 pm, Arrival: 10:00 pm, Airline: British Airways, Duration: 7h 55m
From: London Heathrow  -->  To: New York John F Kennedy Intl

1
$790
8:10 am 8:00 pm
Departure: 8:10 am, Arrival: 8:00 pm, Airline: Delta, Duration: 6h 50m
From: New York John F Kennedy Intl  -->  To: London Heathrow

8:10 pm 11:00 pm
Departure: 8:10 pm, Arrival: 11:00 pm, Airline: Delta, Duration: 7h 50m
From: London Heathrow  -->  To: New York John F Kennedy Intl

2
$429
6:20 pm 6:20 am
Departure: 6:20 pm, Arrival: 6:20 am, Airline: Norse Atlantic UK, Duration: 7h 00m
From: New York John F Kennedy Intl  -->  To: London Gatwick

5:30 pm 1:15 pm
Departure: 5:30 pm, Arrival: 1:15 pm, Airline: Scandinavian Airlines, Duration: 24h 45m
From: London Heathrow  -->  To: Newark

3
$790
6:00 pm 6:25 am
Departure: 6:00 pm, Arrival: 6:2

In [24]:
import pandas as pd
flight_data = []
for i, combo in enumerate(flight_results, start=1):
    flights = combo.find_elements(By.XPATH, ".//li[contains(@class,'hJSA-item')]")
    price = combo.find_elements(By.XPATH, ".//div[contains(@class,'price-section')]//div[contains(@class, 'price-text')]")[0].text

    if len(flights) == 2:
        # Outbound
        flight_out = flights[0]
        out_times = flight_out.find_elements(
            By.XPATH, ".//div[contains(@class,'VY2U')]//span[contains(text(), 'am') or contains(text(), 'pm')]"
        )
        out_dep_time = out_times[0].text.strip().split('+')[0].strip()
        out_arr_time = out_times[1].text.strip().split('+')[0].strip()
        out_airline = flight_out.find_element(
            By.XPATH, ".//div[contains(@class,'c_cgF') and not(.//span)]"
        ).text.strip()
        out_airports = flight_out.find_elements(
            By.XPATH, ".//div[contains(@class,'EFvI')]//div[contains(@class,'c_cgF c_cgF-mod-variant-default')]"
        )
        out_origin = out_airports[0].get_attribute("title")
        out_destination = out_airports[1].get_attribute("title")

        # Return
        flight_ret = flights[1]
        ret_times = flight_ret.find_elements(
            By.XPATH, ".//div[contains(@class,'VY2U')]//span[contains(text(), 'am') or contains(text(), 'pm')]"
        )
        ret_dep_time = ret_times[0].text.strip().split('+')[0].strip()
        ret_arr_time = ret_times[1].text.strip().split('+')[0].strip()
        ret_airline = flight_ret.find_element(
            By.XPATH, ".//div[contains(@class,'c_cgF') and not(.//span)]"
        ).text.strip()
        ret_airports = flight_ret.find_elements(
            By.XPATH, ".//div[contains(@class,'EFvI')]//div[contains(@class,'c_cgF c_cgF-mod-variant-default')]"
        )
        ret_origin = ret_airports[0].get_attribute("title")
        ret_destination = ret_airports[1].get_attribute("title")

        # Append to data list
        flight_data.append({
            "Index": i,
            "Airline Name": out_airline,
            "From": out_origin,
            "To": out_destination,
            "Departure": out_dep_time,
            "Arrival": out_arr_time,
            "Return Airline Name": ret_airline,
            "Return From": ret_origin,
            "Return To": ret_destination,
            "Return Departure": ret_dep_time,
            "Return Arrival": ret_arr_time,
            "Price": price
        })

In [25]:
flight_data

[{'Index': 1,
  'Airline Name': 'British Airways',
  'From': 'New York John F Kennedy Intl',
  'To': 'London Heathrow',
  'Departure': '9:45 am',
  'Arrival': '9:40 pm',
  'Return Airline Name': 'British Airways',
  'Return From': 'London Heathrow',
  'Return To': 'New York John F Kennedy Intl',
  'Return Departure': '7:05 pm',
  'Return Arrival': '10:00 pm',
  'Price': '$959'},
 {'Index': 2,
  'Airline Name': 'Delta',
  'From': 'New York John F Kennedy Intl',
  'To': 'London Heathrow',
  'Departure': '8:10 am',
  'Arrival': '8:00 pm',
  'Return Airline Name': 'Delta',
  'Return From': 'London Heathrow',
  'Return To': 'New York John F Kennedy Intl',
  'Return Departure': '8:10 pm',
  'Return Arrival': '11:00 pm',
  'Price': '$790'},
 {'Index': 3,
  'Airline Name': 'Norse Atlantic UK',
  'From': 'New York John F Kennedy Intl',
  'To': 'London Gatwick',
  'Departure': '6:20 pm',
  'Arrival': '6:20 am',
  'Return Airline Name': 'Scandinavian Airlines',
  'Return From': 'London Heathrow',

In [26]:
df = pd.DataFrame(flight_data)
df.to_csv("flights_output.csv", index=False)
print("✅ Saved to flights_output.csv!")

✅ Saved to flights_output.csv!
